In [1]:
import ccxt
import numpy as np
import pandas as pd

In [2]:
# import ccxt
# exchange = ccxt.okcoin () # default id
# okcoin1 = ccxt.okcoin ({ 'id': 'okcoin1' })
# okcoin2 = ccxt.okcoin ({ 'id': 'okcoin2' })
# id = 'btcchina'
# btcchina = eval ('ccxt.%s ()' % id)
# coinbasepro = getattr (ccxt, 'coinbasepro') ()

# from variable id
exchange_id = 'tokocrypto'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET',
})


In [3]:
markets = exchange.load_markets()
market_list = list(markets.keys())

In [4]:
considered_market = []
for i in market_list:
    if '/USDT' in i:
        considered_market.append(i)

print(len(considered_market))
print(considered_market)

342
['1000SATS/USDT', '1INCH/USDT', 'AAVE/USDT', 'ACA/USDT', 'ACE/USDT', 'ACH/USDT', 'ACM/USDT', 'ADA/USDT', 'ADX/USDT', 'AERGO/USDT', 'AGIX/USDT', 'AGLD/USDT', 'AI/USDT', 'AKRO/USDT', 'ALCX/USDT', 'ALGO/USDT', 'ALICE/USDT', 'ALPACA/USDT', 'ALPHA/USDT', 'ALPINE/USDT', 'ALT/USDT', 'AMP/USDT', 'ANKR/USDT', 'APE/USDT', 'API3/USDT', 'APT/USDT', 'ARB/USDT', 'ARKM/USDT', 'ARPA/USDT', 'AR/USDT', 'ASR/USDT', 'ASTR/USDT', 'ATA/USDT', 'ATM/USDT', 'ATOM/USDT', 'AUCTION/USDT', 'AUDIO/USDT', 'AVAX/USDT', 'AVA/USDT', 'AXL/USDT', 'AXS/USDT', 'BADGER/USDT', 'BAL/USDT', 'BAND/USDT', 'BAR/USDT', 'BAT/USDT', 'BCH/USDT', 'BEAMX/USDT', 'BEL/USDT', 'BETA/USDT', 'BICO/USDT', 'BIFI/USDT', 'BLUR/USDT', 'BLZ/USDT', 'BNB/USDT', 'BNT/USDT', 'BNX/USDT', 'BOND/USDT', 'BONK/USDT', 'BSW/USDT', 'BTC/USDT', 'BTTC/USDT', 'C98/USDT', 'CAKE/USDT', 'CELO/USDT', 'CELR/USDT', 'CFX/USDT', 'CHESS/USDT', 'CHR/USDT', 'CHZ/USDT', 'CITY/USDT', 'CKB/USDT', 'CLV/USDT', 'COMBO/USDT', 'COMP/USDT', 'COS/USDT', 'COTI/USDT', 'CREAM/USDT'

In [5]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

pw = '27111998'
connection = create_server_connection("localhost", "root", pw)

MySQL Database connection successful


In [6]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query, varlist=None):
    cursor = connection.cursor(buffered=True)
    try:
        if varlist is None:
            cursor.execute(query)
        else:
            cursor.execute(query, varlist)
        connection.commit()
        return cursor
    except Error as err:
        print(f"Error: '{err}'")
        return cursor

def executemany_query(connection, query, varlist=None):
    cursor = connection.cursor(buffered=True)
    try:
        if varlist is None:
            cursor.executemany(query)
        else:
            cursor.executemany(query, varlist)
        connection.commit()
        return cursor
    except Error as err:
        print(f"Error: '{err}'")
        return cursor

In [7]:
db = 'crypto_db'
connection = create_db_connection("localhost", "root", pw, db)

MySQL Database connection successful


In [8]:
from datetime import datetime
 
 
timestamp = 1545730073
dt_obj = datetime.fromtimestamp(1709856000000/1000)
 
print("date_time:",dt_obj)
print("type of dt:",type(dt_obj))

date_time: 2024-03-08 07:00:00
type of dt: <class 'datetime.datetime'>


In [26]:
#tes
market = considered_market[2]
market = market.replace('/', '')
print(market)
query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
cur = execute_query(connection,query)
res = cur.fetchall()

df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
df = df.drop_duplicates()
# df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
# day_df = (df.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'}))
# day_df['timestamp'] = day_df.index
init_time = max(df['timestamp'])
final_time = 1710082118000 #8 mar 2024
iter = 0
# # try:
while init_time < final_time:
    test = exchange.fetch_ohlcv (considered_market[2], '1h',since=init_time,limit=1000)
    init_time = test[-1][0]
    test = [tuple(j) for j in test]
    query_string = 'INSERT INTO ' + market + ' (timestamp, open, high, low, close, volume) VALUES (%s,%s,%s,%s,%s,%s)'
    executemany_query(connection,query_string,test)
    # connection.commit()
# except:
#     market_failed.append(market)
#     continue

AAVEUSDT


NetworkError: tokocrypto GET https://api.binance.com/api/v3/klines?interval=1h&limit=1000&symbol=AAVEUSDT&startTime=1709863200000

In [63]:
market_failed = []
for i in range(0,len(considered_market)):
    try:
        market = considered_market[i]
        market = market.replace('/', '')
        print(market)
        query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
        cur = execute_query(connection,query)
        res = cur.fetchall()
        df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df = df.drop_duplicates()
        if not df.empty:
            init_time0 = 0
            init_time = max(df['timestamp'])
            final_time = 1709856000000 #5 mar 2024
            iter = 0
            while init_time < final_time and init_time0 != init_time:
                test = exchange.fetch_ohlcv (considered_market[i], '1h',since=init_time,limit=1000)
                init_time0 = init_time
                init_time = test[-1][0]
                test = [tuple(j) for j in test]
                query_string = 'INSERT INTO ' + market + ' (timestamp, open, high, low, close, volume) VALUES (%s,%s,%s,%s,%s,%s)'
                executemany_query(connection,query_string,test)
            # connection.commit()
    except Error as e:
        print(e)
        market_failed.append(market)
        continue

1000SATSUSDT
Error: '1146 (42S02): Table 'crypto_db.1000satsusdt' doesn't exist'
No result set to fetch from
1INCHUSDT
AAVEUSDT
ACAUSDT
ACEUSDT
ACHUSDT
ACMUSDT
ADAUSDT
ADXUSDT
AERGOUSDT
AGIXUSDT
AGLDUSDT
AIUSDT
AKROUSDT
ALCXUSDT
ALGOUSDT
ALICEUSDT
ALPACAUSDT
ALPHAUSDT
ALPINEUSDT
ALTUSDT
AMPUSDT
ANKRUSDT
APEUSDT
API3USDT
APTUSDT
ARBUSDT
ARKMUSDT
ARPAUSDT
ARUSDT
ASRUSDT
ASTRUSDT
ATAUSDT
ATMUSDT
ATOMUSDT
AUCTIONUSDT
AUDIOUSDT
AVAXUSDT
AVAUSDT
AXSUSDT
BADGERUSDT
BALUSDT
BANDUSDT
BARUSDT
BATUSDT
BCHUSDT
BEAMXUSDT
BELUSDT
BETAUSDT
BICOUSDT
BIFIUSDT
BLURUSDT
BLZUSDT
BNBUSDT
BNTUSDT
BNXUSDT
BONDUSDT
BONKUSDT
BSWUSDT
BTCUSDT
BTTCUSDT
C98USDT
CAKEUSDT
CELOUSDT
CELRUSDT
CFXUSDT
CHESSUSDT
CHRUSDT
CHZUSDT
CITYUSDT
CKBUSDT
CLVUSDT
COMBOUSDT
COMPUSDT
COSUSDT
COTIUSDT
CREAMUSDT
CREOUSDT
CRVUSDT
CTKUSDT
CTSIUSDT
CTXCUSDT
CVPUSDT
CVXUSDT
CYBERUSDT
DARUSDT
DASHUSDT
DATAUSDT
DCRUSDT
DEGOUSDT
DENTUSDT
DEXEUSDT
DFUSDT
DGBUSDT
DIAUSDT
DODOUSDT
DOGEUSDT
DOTUSDT
DUSKUSDT
DYDXUSDT
DYMUSDT
EDUUSDT
EGLDUSDT
ELFUS

[[1694160000000, 0.00253, 0.00256, 0.00252, 0.00252, 105905.0]]

,timestamp,open,high,low,close,volume
0,1646978400000,0.02500,0.04700,0.02500,0.04030,3451300.0
1,1646982000000,0.04027,0.04220,0.03777,0.04219,2095470.0
2,1646985600000,0.04220,0.05182,0.04000,0.04200,6481630.0
3,1646989200000,0.04231,0.04895,0.04030,0.04284,2167960.0
4,1646992800000,0.04284,0.04446,0.03690,0.04100,2161000.0
...,...,...,...,...,...,...
13115,1694145600000,0.00254,0.00254,0.00254,0.00254,0.0
13116,1694149200000,0.00254,0.00254,0.00254,0.00254,0.0
13117,1694152800000,0.00254,0.00257,0.00253,0.00253,235020.0
13118,1694156400000,0.00253,0.00253,0.00253,0.00253,27652.0


In [64]:
exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET',
})

In [65]:
considered_market = []
for i in market_list:
    if '/BTC' in i:
        considered_market.append(i)

print(len(considered_market))
print(considered_market)

263
['1INCH/BTC', 'ACA/BTC', 'ACE/BTC', 'ACH/BTC', 'ADA/BTC', 'ADX/BTC', 'AERGO/BTC', 'AGIX/BTC', 'AGLD/BTC', 'AI/BTC', 'ALCX/BTC', 'ALGO/BTC', 'ALICE/BTC', 'ALPACA/BTC', 'ALPHA/BTC', 'ALPINE/BTC', 'ALT/BTC', 'ANKR/BTC', 'APE/BTC', 'API3/BTC', 'APT/BTC', 'ARB/BTC', 'ARKM/BTC', 'ARPA/BTC', 'AR/BTC', 'AST/BTC', 'ATA/BTC', 'ATOM/BTC', 'AUCTION/BTC', 'AUDIO/BTC', 'AVAX/BTC', 'AVA/BTC', 'AXS/BTC', 'BADGER/BTC', 'BAL/BTC', 'BAND/BTC', 'BAT/BTC', 'BCH/BTC', 'BEL/BTC', 'BETA/BTC', 'BICO/BTC', 'BLUR/BTC', 'BLZ/BTC', 'BNB/BTC', 'BNT/BTC', 'BNX/BTC', 'BOND/BTC', 'C98/BTC', 'CELO/BTC', 'CELR/BTC', 'CFX/BTC', 'CHESS/BTC', 'CHR/BTC', 'CHZ/BTC', 'CLV/BTC', 'COMP/BTC', 'COS/BTC', 'COTI/BTC', 'CRV/BTC', 'CTK/BTC', 'CTSI/BTC', 'CTXC/BTC', 'CYBER/BTC', 'DAR/BTC', 'DASH/BTC', 'DATA/BTC', 'DCR/BTC', 'DEGO/BTC', 'DGB/BTC', 'DIA/BTC', 'DODO/BTC', 'DOGE/BTC', 'DOT/BTC', 'DUSK/BTC', 'DYDX/BTC', 'DYM/BTC', 'EDU/BTC', 'EGLD/BTC', 'ELF/BTC', 'ENJ/BTC', 'ENS/BTC', 'EOS/BTC', 'ETC/BTC', 'ETH/BTC', 'FARM/BTC', 'FET/

In [66]:
market_failed = []
for i in range(0,len(considered_market)):
    try:
        market = considered_market[i]
        market = market.replace('/', '')
        print(market)
        query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
        cur = execute_query(connection,query)
        res = cur.fetchall()
        df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df = df.drop_duplicates()
        if not df.empty:
            init_time0 = 0
            init_time = max(df['timestamp'])
            final_time = 1709856000000 #8 mar 2024
            iter = 0
            while init_time < final_time and init_time0 != init_time:
                test = exchange.fetch_ohlcv (considered_market[i], '1h',since=init_time,limit=1000)
                init_time0 = init_time
                init_time = test[-1][0]
                test = [tuple(j) for j in test]
                query_string = 'INSERT INTO ' + market + ' (timestamp, open, high, low, close, volume) VALUES (%s,%s,%s,%s,%s,%s)'
                executemany_query(connection,query_string,test)
            # connection.commit()
    except Error as e:
        print(e)
        market_failed.append(market)
        continue

1INCHBTC


AuthenticationError: binance {"code":-2008,"msg":"Invalid Api-Key ID."}

In [10]:
exchange.fetch_ohlcv ('ada/usdt', '1h',limit=1000)

BadSymbol: tokocrypto does not have market symbol ada/usdt